# Cheesecake

In [62]:
import pandas as pd
import numpy as np
import csv
import json
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin
import random

In [63]:
import os
os.getcwd()

'C:\\Users\\Admin\\Documents\\Praxis\\Term 3\\CAPP\\NER\\NER_train\\cheesecake'

In [64]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [65]:
# new entity label
LABEL = 'INGREDIENT'

In [ ]:
df = load_data(r"./cheesecake_json.json")


In [66]:
n = int(len())

In [67]:
len(df)

89

In [68]:
df = load_data(r"./cheesecake_json.json")
def convert_data(data):
    final = []
    for item in data:
        text = item["text"]
        ents = item["spans"]
        final_ents = []
        for ent in ents:
            start, end, label = ent["start"], ent["end"], ent["label"]
            final_ents.append([start, end, label])
        ents = {"entities": final_ents}
        final.append([text, ents])
    return (final)
df = convert_data(df)
random.shuffle(df)
ch_train = df[0:65]
ch_valid = df[65:]
write_data(r"./cheesecake_json.json", ch_train)
write_data(r"./cheesecake_json.json", ch_valid)
#print (ch_train)

In [69]:
nlp = spacy.blank("en") 
def create_training(TRAIN_DATA):
    db = DocBin() # create a DocBin object
    for text, annot in tqdm(TRAIN_DATA): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return (db)

In [70]:
ch_train = create_training(ch_train)
ch_train.to_disk("./ch_train.spacy")

ch_valid = create_training(ch_valid)
ch_valid.to_disk("./ch_valid.spacy")

100%|████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 1599.79it/s]

Skipping entity


In [71]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-04-28 20:39:12.469060: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-28 20:39:12.469119: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [72]:
!python -m spacy train config.cfg --output ./output

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     64.00    0.00    0.00    0.00    0.00
 64     200         46.85   1123.24   89.80   88.00   91.67    0.90
132     400          0.00      0.00   89.80   88.00   91.67    0.90
231     600          0.00      0.00   89.80   88.00   91.67    0.90
331     800          0.00      0.00   89.80   88.00   91.67    0.90
436    1000          0.00      0.00   89.80   88.00   91.67    0.90
636    1200          0.00      0.00   89.80   88.00   91.67    0.90
836    1400          0.00      0.00   89.80   88.00 

2022-04-28 20:41:40.730542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-28 20:41:40.730597: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-04-28 20:41:45,382] [INFO] Set up nlp object from config
[2022-04-28 20:41:45,399] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-28 20:41:45,403] [INFO] Created vocabulary
[2022-04-28 20:41:45,406] [INFO] Finished initializing nlp object
[2022-04-28 20:41:45,609] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [81]:
test_data = load_data("./test_data.json")

In [82]:
len(test_data)

28

In [83]:
print(test_data[1])

{'text': '125g butter melted', 'spans': [{'start': 5, 'end': 11, 'label': 'INGREDIENT'}]}


In [84]:
test_data[1]['text']

'125g butter melted'

In [85]:
nlp = spacy.load("output/model-best")

In [86]:
doc = nlp(test_data[1]['text'])

In [87]:
doc = nlp(test_data[0]['text'])
for ent in doc.ents:
    print(ent.text,ent.label_)

packet INGREDIENT
biscuits INGREDIENT


In [88]:
j = 0
for i in test_data:
    print("test_data:"+ str(j))
    doc= nlp(i['text'])
    for ent in doc.ents:
        print(ent.text,ent.label_,ent.start_char,ent.end_char)
    j += 1

test_data:0
packet INGREDIENT 5 11
biscuits INGREDIENT 24 32
test_data:1
butter INGREDIENT 5 11
test_data:2
cream cheese INGREDIENT 17 29
test_data:3
caster sugar INGREDIENT 8 20
test_data:4
cream INGREDIENT 13 18
test_data:5
vanilla extract INGREDIENT 6 21
test_data:6
finely INGREDIENT 6 12
lemon INGREDIENT 20 25
test_data:7
raspberries INGREDIENT 7 18
test_data:8
digestive INGREDIENT 10 19
test_data:9
butter INGREDIENT 20 26
test_data:10
sugar INGREDIENT 18 23
test_data:11
cream INGREDIENT 13 18
test_data:12
starch INGREDIENT 18 24
test_data:13
sugar INGREDIENT 28 33
test_data:14
cream cheese INGREDIENT 7 19
test_data:15
egg INGREDIENT 2 5
test_data:16
vanilla extract INGREDIENT 11 26
test_data:17
lemon INGREDIENT 13 18
test_data:18
graham cracker INGREDIENT 7 21
test_data:19
butter INGREDIENT 4 10
test_data:20
sugar INGREDIENT 10 15
test_data:21
cream cheese INGREDIENT 19 31
test_data:22
eggs INGREDIENT 2 6
test_data:23
vanilla extract INGREDIENT 13 28
test_data:24
sugar INGREDIENT 